In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from unicodedata import normalize
from nltk.stem import SnowballStemmer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **PROCESO DE ENTRENAMIENTO DE ALGORITMOS**

Antes de entrenar un algoritmo, ***recabaremos información del csv*** para comparar los datos iniciales con los finales. 
Debemos limpiar los datos que con los que este va a trabajar y presentarlos en el formato adecuado para este algoritmo, pero antes, los datos preprocesados deben formar parte de la bolsa de palabras con la que trabajará el algoritmo.

Para posteriormente poder comprobar la reducción de palabras y letras después del preprocesado ***añadimos dos columnas*** al dataframe inicial, donde la ***primera*** corresponde al ***número de palabras*** que hay por línea y la ***segunda*** el ***número de caracteres***.

In [ ]:
def csv_len(data) :
    
    words = []
    letters = []
    sentiments = []
    tweets = []
    
    for index, tweet in data.iterrows():
        tweet_split = tweet.Tweet.split()
        
        sentiments.append(tweet.Sentiment)
        tweets.append(tweet.Tweet)
        letters.append(len(tweet.Tweet))
        words.append(len(tweet_split))
    
    data['Tweet'] = tweets
    data['Sentiment'] = sentiments
    data['Words'] = words
    data['Letters'] = letters
    return data

A través de los datos recogidos en la función csv_len ***mostraremos un gráfico*** donde ***muestra los rangos de cantidad de palabras y letras*** con las que estamos trabajando, haciendo así más visuales los datos recogidos.

In [ ]:
def graphic(data_len) :
    
    fig,ax = plt.subplots(figsize=(5,5))
    plt.boxplot(data_len)
    plt.show()

***En el preprocesamiento: *** 

1. Eliminaremos ****enlaces*** (comienzan por http o https y terminan por .com o .es), ***menciones***, ***hashtags*** y ***retweets***.
El siguiente paso sería eliminar: 
1. Los ***caracteres duplicados***, en este caso se contempla que si un caracter se repite 3 veces se sustituirá por una sola ocurrencia de este.
2. Las ***risas*** (hahaha, jajaja) contemplando que puedan ser con cualquier vocal.
3. Eliminamos las tildes o caracteres de raros que se encuentren en las letras.
4. Eliminamos cualquier ***caracter que no sean letras***, ya que estos no tienen ningún valor.
5. Se sustituyen los ***tabuladores*** por espacios en blanco.
6. Si hay varios ***espacios en blanco*** se sustituyen por ***un solo espacio***.
7. ***Eliminamos espacios*** al comienzo del tweet y al final.
8. ***Eliminamos los saltos de línea*** que puedan existir en cualquier parte de la cadena.
9. Eliminamos ***Stopwords***: es decir eliminamos la palabras vacías que por sí solas no tienen ningún tipo de significado, sino que sirven para acompañar o modificar a otras palabras.
10. Reducimos las palabras presentes a su ***raíz*** correspondiente.

In [ ]:
def preprocessing(data) :
    
    tweets = []
    sentiment = []

    for index, tweet in data.iterrows():
        words_cleaned=""
        tweet_clean = tweet.Tweet.lower()
    
        words_cleaned =" ".join([word for word in tweet_clean.split()
            if 'http://' not in word
            and 'https://'not in word
            and '.com' not in word
            and '.es' not in word
            and not word.startswith('@')
            and not word.startswith('#')
            and word != 'rt'])
        
        
        tweet_clean = re.sub(r'\b([jh]*[aeiou]*[jh]+[aeiou]*)*\b',"",words_cleaned)
        tweet_clean = re.sub(r'(.)\1{2,}',r'\1',tweet_clean)
        tweet_clean = re.sub(
            r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", 
            normalize( "NFD", tweet_clean), 0, re.I)
        tweet_clean = re.sub("[^a-zA-Z]"," ",tweet_clean)
        tweet_clean = re.sub("\t", " ", tweet_clean)
        tweet_clean = re.sub(" +", " ",tweet_clean) 
        tweet_clean = re.sub("^ ", "", tweet_clean)
        tweet_clean = re.sub(" $", "", tweet_clean)
        tweet_clean = re.sub("\n", "", tweet_clean)
        
        words_cleaned=""
        stemmed =""
        
        stop_words = set(stopwords.words('english'))
        stemmer = SnowballStemmer('english')
        
        tokens = word_tokenize(tweet_clean)
        
        words_cleaned =[word for word in tokens if not word in stop_words]
        stemmed = " ".join([stemmer.stem(word) for word in words_cleaned])
        
        
    
        sentiment.append(tweet.Sentiment)
        tweets.append(stemmed)
    
    data['Tweet'] = tweets
    data['Sentiment'] = sentiment
    data.loc[:,['Sentiment','Tweet']]
    
    return data

# **Corpus: *Sem-Eval ***

En primer lugar obtenemos el dataframe de entrenamiento.

In [ ]:
train = pd.read_csv('/kaggle/input/semevaldatadets/semeval-2017-train.csv', delimiter='	')
train.columns = ['Sentiment', 'Tweet']
train.rename(columns={'label': 'Sentiment','text' : 'Tweet'})

En segundo lugar obtenemos el dataframe de predicción.

In [ ]:
test = pd.read_csv('/kaggle/input/semevaldatadets/semeval-2017-test.csv', delimiter='	')
test.columns = ['Sentiment', 'Tweet']
test.rename(columns={'label': 'Sentiment','text' : 'Tweet'})

**1. Recogida de información.**

Comprobamos los diferentes valores de sentimientos que hay tanto en el dataframe test como en el train. En este caso hay tres (-1, 0, 1) donde cada uno tiene un valor de filas asignado. 

In [ ]:
train.Sentiment.value_counts()

In [ ]:
test.Sentiment.value_counts()

Obtenemos la ***palabras y letras por tweet*** del dataframe train antes del preprocesado.

In [ ]:
train = csv_len(train)
train

Mostramos el ***gráfico*** asociado a las ***palabras*** por tweet antes del preprocesamiento.

In [ ]:
graphic(train['Words'])

Mostramos el ***gráfico*** asociado a las ***letras*** por tweet antes del preprocesamiento.

In [ ]:
graphic(train['Letters'])

Obtenemos la ***palabras y letras por tweet*** del dataframe test antes del preprocesado.

In [ ]:
test = csv_len(test)
test

Mostramos el ***gráfico*** asociado a las ***palabras*** por tweet antes del preprocesamiento.

In [ ]:
graphic(test['Words'])

Mostramos el ***gráfico*** asociado a las ***letras*** por tweet antes del preprocesamiento.

In [ ]:
graphic(test['Letters'])

**2. Preprocesamiento de datos.** 

Limpiamos los datos con los que trabajará el algoritmo utilizando la función ***"preprocessing"***.

In [ ]:
train_cleaned = preprocessing(train)
train_cleaned.loc[:,['Sentiment','Tweet']]

In [ ]:
test_cleaned = preprocessing(test)
test_cleaned.loc[:,['Sentiment','Tweet']]

**2. Guardamos el preprocesaiento.** 

Guardamos los datos preprocesados en un csv para poder descargarlos y utilizarlos en un notebook para entrenar los clasificadores.

In [ ]:
train_final = train_cleaned.loc[:,['Sentiment','Tweet']]
train_final
train_final.to_csv('semevalTrain.csv',index=False)

In [ ]:
test_final = test_cleaned.loc[:,['Sentiment','Tweet']]
test_final
test_final.to_csv('semevalTest.csv',index=False)

Obtenemos la ***palabras y letras por tweet*** del dataframe train después del preprocesado.

In [ ]:
train_cleaned = csv_len(train_cleaned)
train_cleaned

Mostramos el ***gráfico*** asociado a las ***palabras*** por tweet después del preprocesamiento.

In [ ]:
graphic(train_cleaned['Words'])

Mostramos el ***gráfico*** asociado a las ***letra*** por tweet después del preprocesamiento.

In [ ]:
graphic(train_cleaned['Letters'])

Obtenemos la ***palabras y letras por tweet*** del dataframe test después del preprocesado.

In [ ]:
test_cleaned = csv_len(test_cleaned)
test_cleaned

Mostramos el ***gráfico*** asociado a las ***palabras*** por tweet después del preprocesamiento.

In [ ]:
graphic(test_cleaned['Words'])

Mostramos el ***gráfico*** asociado a las ***letra*** por tweet después del preprocesamiento.

In [ ]:
graphic(test_cleaned['Letters'])